In [ ]:
# Black-Scholes Streaming Example
import tributary.streaming as ts
import math


# Define a standard normal CDF
def normal_cdf(x):
    return ts.Div(
        ts.Add(
            ts.Erf(ts.Div(x, ts.Const((math.sqrt(2),0), use_dual=True))), 
            ts.Const((1,0), use_dual=True)
        ),
        ts.Const((2,0), use_dual=True)
    )


# Define Inputs
def _time():
    for _ in [1, 2, 3, 4, 5, 6, 7]:
        yield (_, 1)

strike_price = ts.Const((203,0), use_dual=True)
stock_price = ts.Const((210,0), use_dual=True)
r = ts.Const((0.2175, 0), use_dual=True)
time = ts.Div(ts.Node(foo=_time, name='Time', use_dual=True), ts.Const((252,0), use_dual=True))
vol = ts.Const((14.04, 0), use_dual=True)
   

# Define Equation
'''
Option Price = Stock_Price * Normal(d1) - Strike_price * e ^(-rate * time) * Normal(d2)
d1 = (ln(Stock_price/Strike_price) + (rate + vol^2/2) * time ) / vol * sqrt(time)
d2 = d1 - vol * sqrt(time) 
'''
d1 = ts.Div(
            ts.Add(ts.Log(ts.Div(stock_price, strike_price)),
                   ts.Mult(time, 
                         ts.Add(r, 
                                ts.Div(ts.Pow(vol,2),
                                       ts.Const((2,0), use_dual=True)
                                       )
                                )
                        )
                ),
            ts.Mult(vol, ts.Sqrt(time))
)

d2 = ts.Sub(d1, ts.Mult(vol, ts.Sqrt(time)))

opt_price = ts.Sub(ts.Mult(stock_price, normal_cdf(d1)),
                   ts.Mult(strike_price, ts.Mult(
                                                 ts.Exp(ts.Negate(ts.Mult(r,time))),
                                                 normal_cdf(d2)
                                                 )
                            )
                    )

# Run and get time derivative
out = ts.run(opt_price)
thetas = [x[1] for x in out]

In [ ]:
# Black-Scholes Streaming Example
import tributary.lazy as tl
import math


# Define Inputs
def normal_cdf_lazy(x):
    return tl.Div(
        tl.Add(
            tl.Erf(tl.Div(x, (math.sqrt(2),0))), 
            (1,0)
        ),
        tl.Node((2,0), use_dual=True)
    )


strike_price = tl.Node(value=(203, 0), name='Strike Price', use_dual=True)
stock_price = tl.Node(value=(210, 0), name='Stock Price', use_dual=True)
r = tl.Node(value=(0.2175, 0), name='Risk Free Interest Rate', use_dual=True)
time = tl.Div(tl.Node(value=(4, 1), name='Time to Maturity', use_dual=True), 
              tl.Node(value=(252,0), use_dual=True))
vol = tl.Node(value=(14.04, 0), name='Stock Price Volatility', use_dual=True)


# Define Equation
'''
Option Price = Stock_Price * Normal(d1) - Strike_price * e ^(-rate * time) * Normal(d2)
d1 = (ln(Stock_price/Strike_price) + (rate + vol^2/2) * time ) / vol * sqrt(time)
d2 = d1 - vol * sqrt(time) 
'''

d1 = tl.Div(
            tl.Add(tl.Log(tl.Div(stock_price,strike_price)),
                 tl.Mult(time, 
                         tl.Add(r, 
                                tl.Div(tl.Pow(vol,2),
                                       tl.Node((2,0), use_dual=True)
                                       )
                                )
                        )
                ),
            tl.Mult(vol, tl.Sqrt(time))
)

d2 = tl.Sub(d1, tl.Mult(vol, tl.Sqrt(time)))

opt_price = tl.Sub(tl.Mult(stock_price, normal_cdf_lazy(d1)),
                   tl.Mult(strike_price, tl.Mult(
                                                 tl.Exp(tl.Negate(tl.Mult(r,time))),
                                                 normal_cdf_lazy(d2)
                                                 )
                            )
                    )


# Run and get time derivative
out = opt_price()